<a href="https://colab.research.google.com/github/rafavidal1709/projeto-aplicado-iii/blob/main/Acur%C3%A1cia_dos_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregando o dataset de avaliação

Vamos conectar ao Google Drive para salvar e carregar arquivos permanentes.

Também vamos criar funções para lidar com o carregamento, armazenamento e transformação do dataset em seu formato original.

In [ ]:
from google.colab import drive
import os
import json
from google.colab import files

drive.mount('/content/drive')

# Caminho para a pasta no Google Drive
folder_path = '/content/drive/MyDrive/projeto-aplicado-iii'

# Verificar se a pasta existe, se não, criar
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Faz o upload de um dataset
def upload_dataset():
  uploaded = files.upload()
  for file_name in uploaded.keys():
      with open(file_name, 'r', encoding='utf-8') as f:
          data = json.load(f)
  return data, file_name

# Formata o dataset original para o formato desejado
def format_dataset(data):
  dataset = {'text':[],'category':[],'embedding':{},'accuracy':{}}
  for c in range(len(data)):
    for i in data[c]['examples']:
      dataset['text'].append(i)
      dataset['category'].append(data[c]['category'])
  return dataset

# Salva o dataset no Google Drive
def save_dataset(dataset, filename):
  file_path = os.path.join(folder_path, filename+'.json')
  with open(file_path, 'w') as f:
    json.dump(dataset, f)

# Executa a sequência das três funções anteriores
def full_upload_dataset():
  data, file_name = upload_dataset()
  dataset = format_dataset(data)
  save_dataset(dataset,file_name)
  return dataset

# Carrega o dataset do Google Drive
def load_dataset(filename):
  file_path = os.path.join(folder_path, filename+'.json')
  with open(file_path, 'r') as f:
    data = json.load(f)
  if 'embedding' in data:
    data['embedding'] = {int(k): v for k, v in data['embedding'].items()}
  return data

Mounted at /content/drive


Se o dataset ainda não foi baixado e salvo no Google Drive:

In [ ]:
dataset = full_upload_dataset()

Saving Dataset 100 new.json to Dataset 100 new.json


OU se ele já está no formato correto no Google Drive:

In [ ]:
dataset = load_dataset("dataset_accuracy")
dataset2 = load_dataset("dataset_accuracy_2")
accuracy = load_dataset("evaluate")
mask = load_dataset("mask")

Também pode salvá-lo no Google Drive:

In [ ]:
#save_dataset(dataset, "dataset_accuracy")
#save_dataset(dataset2, "dataset_accuracy_2")

# Métrica de acurácia 1

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def accuracy_test_cosine(dataset, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    for i in range(len(dataset['text'])):
        # Calculando similaridade cosseno entre o embedding atual e todos os outros embeddings
        similarities = cosine_similarity([embeddings[i]], embeddings)
        ranked_indices = np.argsort(similarities[0])[::-1]  # Ordenar por similaridade

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):
            if dataset['category'][i] == dataset['category'][ranked_indices[j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def accuracy_test_knn(dataset, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings = np.array(embeddings)

    # Usando NearestNeighbors para calcular os k vizinhos mais próximos (k=top_n)
    knn = NearestNeighbors(n_neighbors=len(dataset['text']), metric='euclidean')
    knn.fit(embeddings)

    for i in range(len(dataset['text'])):
        # Encontrando os k vizinhos mais próximos para o embedding atual
        distances, ranked_indices = knn.kneighbors([embeddings[i]])

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):  # Considerando apenas os top_n vizinhos
            if dataset['category'][i] == dataset['category'][ranked_indices[0][j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

# Avaliação dos modelos 1



Primeiro aplicamos a similaridade de cossenos para medir a acurácia de encontarmos os textos mais similares no sistema de recomendação. Os textos da mesma categoria têm um teor semelhante, sendo assim deveriam ser colocados mais próximos no hiperplano do embedding.

Como métrica vamos encontrar os oito textos mais similares que pertencem a mesma categoria e somar seus índices. Na situação ideal deveria ser: 0, 1, 2, 3, 4, 5, 6, 7; o que soma 28. Por isso a diferença entre o valor obtido com o ideal define o quão impreciso é o modelo, sendo assim quanto mais longe de zero pior é o modelo utilizado.

Para cada modelo temos a média (mean), a média por categoria e por último a matriz resultante de todas as iterações do processo. Cada modelo também cria dois tipos de embedding o 'pooler output' e o 'output mean', ambos são abordagens comuns utilizadas nessa situação.

Na comparação entre os dois modelo multilinguas - BERT e Longformer -, o BERT apresentou melhor desempenho com 44.7 contra 82.43, representando um desempenho quase duas vezes melhor. Já o BERTimbau, que já é treinado em português, conseguiu uma média de 3.6, superando todos os demais modelos.

In [ ]:
dataset['accuracy']['cosine_similarity']={}
for emb in dataset['embedding'][0].keys():
  for key, value in accuracy_test_cosine(dataset, emb).items():
    dataset['accuracy']['cosine_similarity'][key] = value

display(dataset['accuracy']['cosine_similarity'])

{'longformer_global_attention': {'mean': 103.3,
  'mean_per_cat': {'queimadas': 92.5,
   'trabalhadores_sem_epi': 81.2,
   'pisoteamento_nascente': 116.25,
   'agrotoxico_proximo_residencias': 105.3,
   'contaminacao_nascente_agrotoxico': 121.25},
  'matrix': [[0.0, 1.0, 2.0, 7.0, 8.0, 10.0, 12.0, 15.0],
   [0.0, 2.0, 3.0, 6.0, 8.0, 12.0, 13.0, 14.0],
   [0.0, 7.0, 22.0, 25.0, 30.0, 36.0, 39.0, 47.0],
   [0.0, 2.0, 3.0, 5.0, 11.0, 13.0, 15.0, 16.0],
   [0.0, 8.0, 19.0, 24.0, 34.0, 43.0, 51.0, 63.0],
   [0.0, 20.0, 29.0, 47.0, 50.0, 57.0, 63.0, 69.0],
   [0.0, 2.0, 7.0, 11.0, 13.0, 14.0, 15.0, 19.0],
   [0.0, 1.0, 4.0, 6.0, 8.0, 11.0, 14.0, 15.0],
   [0.0, 2.0, 4.0, 5.0, 7.0, 12.0, 16.0, 18.0],
   [0.0, 5.0, 10.0, 11.0, 12.0, 14.0, 18.0, 24.0],
   [0.0, 4.0, 7.0, 9.0, 11.0, 13.0, 16.0, 17.0],
   [0.0, 3.0, 6.0, 11.0, 13.0, 14.0, 15.0, 19.0],
   [0.0, 3.0, 8.0, 9.0, 11.0, 12.0, 15.0, 16.0],
   [0.0, 3.0, 14.0, 19.0, 27.0, 30.0, 35.0, 39.0],
   [0.0, 6.0, 7.0, 10.0, 11.0, 14.0, 20.0, 25.0

A segunda métrica, muito mais leve de ser calculada é o KNN. O resultado abaixo segue o mesmo padrão do anterior com média, média por categoria e matrix resultante.

Se compararmos com o cossine similarity podemos concluir que os resultados são praticmante identicos tendo apenas pequenas variações insignificantes, o que nos faz escolher pelo KNN pela significante maior rapidez em realizar os cálculos com este método.

In [ ]:
dataset['accuracy']['knn_euclidian']={}
for emb in dataset['embedding'][0].keys():
  for key, value in accuracy_test_knn(dataset, emb).items():
    dataset['accuracy']['knn_euclidian'][key] = value

display(dataset['accuracy']['knn_euclidian'])

{'longformer_global_attention': {'mean': 103.3,
  'mean_per_cat': {'queimadas': 92.5,
   'trabalhadores_sem_epi': 81.2,
   'pisoteamento_nascente': 116.25,
   'agrotoxico_proximo_residencias': 105.3,
   'contaminacao_nascente_agrotoxico': 121.25},
  'matrix': [[0.0, 1.0, 2.0, 7.0, 8.0, 10.0, 12.0, 15.0],
   [0.0, 2.0, 3.0, 6.0, 8.0, 12.0, 13.0, 14.0],
   [0.0, 7.0, 22.0, 25.0, 30.0, 36.0, 39.0, 47.0],
   [0.0, 2.0, 3.0, 5.0, 11.0, 13.0, 15.0, 16.0],
   [0.0, 8.0, 19.0, 24.0, 34.0, 43.0, 51.0, 63.0],
   [0.0, 20.0, 29.0, 47.0, 50.0, 57.0, 63.0, 69.0],
   [0.0, 2.0, 7.0, 11.0, 13.0, 14.0, 15.0, 19.0],
   [0.0, 1.0, 4.0, 6.0, 8.0, 11.0, 14.0, 15.0],
   [0.0, 2.0, 4.0, 5.0, 7.0, 12.0, 16.0, 18.0],
   [0.0, 5.0, 10.0, 11.0, 12.0, 14.0, 18.0, 24.0],
   [0.0, 4.0, 7.0, 9.0, 11.0, 13.0, 16.0, 17.0],
   [0.0, 3.0, 6.0, 11.0, 13.0, 14.0, 15.0, 19.0],
   [0.0, 3.0, 8.0, 9.0, 11.0, 12.0, 15.0, 16.0],
   [0.0, 3.0, 14.0, 19.0, 27.0, 30.0, 35.0, 39.0],
   [0.0, 6.0, 7.0, 10.0, 11.0, 14.0, 20.0, 25.0

# Métrica de acurácia 2

Após o sucesso do primeiro teste quisemos testar o modelo em situações realistas. Os exemplos utilizados acima são denúncias ambientais que foram limpos para compor a base de dados de referência. Para não haver tendenciosidade em misturar fatores como nome de indivíduos e locais com o teor do que está sendo reportado, essas informações foram limpas do texto.

Já abaixo vamos continuar utilizando a mesma base de dados como referência, porém para medir a similaridade com outro conjunto de dados rotulado nas mesmas cinco categorias e com nomes e locais.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def accuracy2_test_cosine(dataset, dataset2, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset2['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    embeddings2 = []
    for i in range(len(dataset2['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings2.append(dataset2['embedding'][i][embedding])

    for i in range(len(dataset2['text'])):
        # Calculando similaridade cosseno entre o embedding atual e todos os outros embeddings
        similarities = cosine_similarity([embeddings2[i]], embeddings)
        ranked_indices = np.argsort(similarities[0])[::-1]  # Ordenar por similaridade

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):
            if dataset2['category'][i] == dataset['category'][ranked_indices[j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset2['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset2['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def accuracy2_test_knn(dataset, dataset2, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset2['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings = np.array(embeddings)

    embeddings2 = []
    for i in range(len(dataset2['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings2.append(dataset2['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings2 = np.array(embeddings2)

    # Usando NearestNeighbors para calcular os k vizinhos mais próximos (k=top_n)
    knn = NearestNeighbors(n_neighbors=len(dataset['text']), metric='euclidean')
    knn.fit(embeddings)

    for i in range(len(dataset2['text'])):
        # Encontrando os k vizinhos mais próximos para o embedding atual
        distances, ranked_indices = knn.kneighbors([embeddings2[i]])

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):  # Considerando apenas os top_n vizinhos
            if dataset['category'][i] == dataset['category'][ranked_indices[0][j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset2['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset2['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

# Avaliação dos modelos 2

Ao adicionar essas informações vemos uma queda vertiginosa na acurácia, tendo o melhor resultado sendo 130.04 utiilizando ainda o BERTimbau, porém o modelo multilinguas do BERT se equiparou ao modelo Longformer. Nesse primeiro contexto da nova avaliação feita, muitas da confiança que tinhamos no método é colocada em dúvida novamente.

In [ ]:
dataset2['accuracy']['cosine_similarity']={}
for emb in dataset2['embedding'][0].keys():
  for key, value in accuracy2_test_cosine(dataset, dataset2, emb).items():
    dataset2['accuracy']['cosine_similarity'][key] = value

display(dataset2['accuracy']['cosine_similarity'])

{'longformer_global_attention': {'mean': 153.52,
  'mean_per_cat': {'queimadas': 93.4,
   'trabalhadores_sem_epi': 141.6,
   'pisoteamento_nascente': 125.3,
   'agrotoxico_proximo_residencias': 210.5,
   'contaminacao_agua_agrotoxico': 196.8},
  'matrix': [[6.0, 8.0, 29.0, 36.0, 42.0, 49.0, 54.0, 63.0],
   [8.0, 20.0, 31.0, 42.0, 53.0, 57.0, 60.0, 67.0],
   [0.0, 3.0, 4.0, 11.0, 13.0, 14.0, 15.0, 19.0],
   [1.0, 7.0, 14.0, 16.0, 23.0, 26.0, 28.0, 32.0],
   [7.0, 9.0, 11.0, 31.0, 44.0, 47.0, 53.0, 66.0],
   [3.0, 5.0, 8.0, 20.0, 23.0, 24.0, 32.0, 37.0],
   [2.0, 25.0, 30.0, 34.0, 45.0, 53.0, 60.0, 68.0],
   [1.0, 22.0, 26.0, 35.0, 48.0, 55.0, 60.0, 68.0],
   [1.0, 14.0, 34.0, 35.0, 52.0, 54.0, 59.0, 68.0],
   [8.0, 10.0, 11.0, 20.0, 24.0, 25.0, 26.0, 41.0],
   [17.0, 18.0, 22.0, 26.0, 46.0, 48.0, 49.0, 53.0],
   [12.0, 20.0, 38.0, 39.0, 41.0, 43.0, 46.0, 48.0],
   [8.0, 30.0, 35.0, 37.0, 39.0, 44.0, 48.0, 51.0],
   [1.0, 3.0, 8.0, 17.0, 19.0, 29.0, 31.0, 36.0],
   [0.0, 6.0, 24.0, 26.0,

Porém voltamos a melhorar o desempenho ao aplicar o KNN em vez da similaridade de cossenos, o BERT volta a se destacar sobre o Longformer. O melhor resultado obtido, desta vez, foi o BERT multilingua (com pooler output) com 85.56 contra 109.96 do BERTimbau (com output mean).

In [ ]:
dataset2['accuracy']['knn_euclidian']={}
for emb in dataset2['embedding'][0].keys():
  for key, value in accuracy2_test_knn(dataset, dataset2, emb).items():
    dataset2['accuracy']['knn_euclidian'][key] = value

display(dataset2['accuracy']['knn_euclidian'])

{'longformer_global_attention': {'mean': 187.16,
  'mean_per_cat': {'queimadas': 196.6,
   'trabalhadores_sem_epi': 128.6,
   'pisoteamento_nascente': 188.8,
   'agrotoxico_proximo_residencias': 210.6,
   'contaminacao_agua_agrotoxico': 211.2},
  'matrix': [[6.0, 8.0, 29.0, 36.0, 42.0, 49.0, 54.0, 63.0],
   [8.0, 20.0, 31.0, 42.0, 53.0, 57.0, 60.0, 67.0],
   [0.0, 3.0, 4.0, 11.0, 13.0, 14.0, 15.0, 19.0],
   [1.0, 7.0, 14.0, 16.0, 23.0, 26.0, 28.0, 32.0],
   [7.0, 9.0, 11.0, 31.0, 44.0, 47.0, 53.0, 66.0],
   [3.0, 5.0, 8.0, 20.0, 23.0, 25.0, 32.0, 37.0],
   [2.0, 25.0, 30.0, 34.0, 45.0, 53.0, 60.0, 68.0],
   [1.0, 22.0, 26.0, 35.0, 48.0, 55.0, 60.0, 68.0],
   [1.0, 14.0, 34.0, 35.0, 52.0, 54.0, 59.0, 68.0],
   [8.0, 10.0, 11.0, 20.0, 24.0, 25.0, 26.0, 41.0],
   [4.0, 19.0, 27.0, 39.0, 44.0, 52.0, 61.0, 68.0],
   [3.0, 13.0, 30.0, 31.0, 36.0, 53.0, 61.0, 68.0],
   [3.0, 24.0, 28.0, 36.0, 41.0, 53.0, 61.0, 68.0],
   [0.0, 6.0, 7.0, 9.0, 10.0, 12.0, 15.0, 20.0],
   [5.0, 7.0, 18.0, 29.0, 3

# Conclusão

Testamos os métodos Longformer e BERT da arquitetura transformers e pudemos concluir que o BERT e o BERTimbau são melhores para lidar com a tarefa de embedding. Porém, pelos resultados que temos até o momento, seria preciso uma análise mais profunda da versão especializada em português e multilingua do BERT para definir qual seria a mais apropriada.

Mesmo para decidir entre o método de embedding pooler output e output mean, não podemos definir com exatidão o melhor. Porém o melhor desempenho registrado ao tratar dados mais realistas foi utilizando o pooler output do modelo multilinguas do BERT.